In [10]:
from lyricsgenius import Genius
import os

In [11]:
token = "EaFwVotR7TKt9kDiKTnUmjgEciv5vFvoljxqArZ3Sf4pj0BVFEzpTojPqPo7FTFh"
genius = Genius(token)
genius.skip_non_songs = True
genius.timeout = 10
genius.retries = 3

In [12]:
import pickle 
def check_artist_path(artist, override = False):
    artist_f_id = '-'.join(artist.lower().split(' '))
    if os.path.exists(f'./artists/{artist_f_id}.p') and not override:
        print("Lyrics have already been created for this artist")
        return f'./artists/{artist_f_id}.p'
    return None
artist_path = check_artist_path('Taylor Swift')

# def check_artist_songs(artist):
#     artist_f_id = '-'.join(artist.lower().split(' '))
#     songs_path = f'./artists/{artist_f_id}_songs.p'
#     if os.path.exists(songs_path):
#         print("Songs have already been collected for this artist. Loading those...")
#         with open(songs_path, 'rb') as handle:
#             return pickle.load(handle)
#     return None

In [13]:
def get_all_songs(artist):
    artist_found = genius.search_artist(artist, max_songs = 5)
    if artist_found.name.lower() != artist.lower():
        print(f"Please note that the artist requested was {artist}")
        print(f"However, the artist used to find search results is {artist_found.name}")
        print(f"If this seems incorrect, please go back to source code and review!")
    model_name = artist_found.url[artist_found.url.rfind('/') + 1:].lower()
    print(artist_found.id)
    return artist_found, model_name
ls = get_all_songs('Laura Stevenson')


Searching for songs by Laura Stevenson...

Song 1: "Living Room, NY"
Song 2: "Jellyfish"
Song 3: "Torch Song"
Song 4: "Master of Art"
Song 5: "Dermatillomania"

Reached user-specified song limit (5).
Done. Found 5 songs.
173907


In [14]:
def limit_songs(artist_found):
    song_list = artist_found.songs
    print(len(song_list))
    artist_found.songs = [song for song in song_list if artist_found.name.lower() in song.primary_artist.name.lower()]
    print(len(song_list))
    return artist_found
# swift2 = limit_songs(swift)

In [15]:
import re
def standardize_song_name(name):
    a = re.sub(r'\([^)]*\)', '', name)
    b = re.sub(r'\[[^)]*\]', '', a)
    return b.strip().lower().replace('\u200b','')

def clean_lyrics(lyrics):
    lyrics = re.sub(r'(\[.*?\])*', '', lyrics)
    lyrics = re.sub('\n{2}', '\n', lyrics)  # Gaps between verses
    lyrics = re.sub('\nYou might also like\n','', lyrics)

    # Want to get consistent apostrophes to get consistent treatment of contractions
    lyrics = str(lyrics).replace("'", "’")
    
    lyrics = str(lyrics.strip("\n"))
    lyrics = lyrics.replace("EmbedShare URLCopyEmbedCopy", "")
    lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics)
    lyrics = re.sub(r'\d+$', '', lyrics)
    lyrics = str(lyrics).lstrip().rstrip()
    lyrics = str(lyrics).replace("\n\n", "\n")
    lyrics = str(lyrics).replace("\n\n", "\n")
    lyrics = re.sub(' +', ' ', lyrics)
    lyrics = str(lyrics).replace('"', "")
    # lyrics = str(lyrics).replace("'", "")
    lyrics = str(lyrics).replace("*", "")

    # Remove text at front of lyrics, claiming this is start of lyrics
    lyrics = re.sub('(^[\s\S]* Lyrics[\\n]*)','', lyrics)
    lyrics = re.sub('(^[\s\S]* Lyrics\\n)','', lyrics)
    if 'lyrics' in lyrics[:100].lower():
        print("Lyrics are not properly cleaned!")
    
    # Remove some random symbols that represent spaces
    lyrics = re.sub('\u2005',' ', lyrics)
    
    # Remove non-lyrics text that sometimes appears in lyrics
    lyrics = re.sub('\d*Embed$', '', lyrics)
    lyrics = re.sub('(You might also like$)','', lyrics)
    if 'You might also like' in lyrics:
        print("Check out lyrics here...")
        print(lyrics)
        print()
    return lyrics
    
def get_lyrics(artist_found):
    newDict = {}
    for song in artist_found.songs:
        title = standardize_song_name(song.title)
        if title not in newDict.keys():
            newDict[title] = clean_lyrics(song.lyrics)
    return newDict
# lyric_dict = get_lyrics(swift2)

In [16]:
import pickle
def write_pickle(artist, lyric_dict):
    artist_f_id = '-'.join(artist.lower().split(' '))
    with open(f'./artists/{artist_f_id}.p', 'wb') as handle:
        pickle.dump(lyric_dict, handle)

In [17]:
def get_and_save_artist_lyrics(artist, override = False):
    """
    Returns:
    -A lyrics dict with
        title : lyric 
      pair for all songs found under the artist
    """
    artist_path = check_artist_path(artist, override = override)
    if artist_path and not override:
        with open(artist_path, 'rb') as handle:
            return pickle.load(handle)
    
    print(f"Getting songs for {artist}...")
    artist_found, model_name = get_all_songs(artist)
    print("Limiting songs...")
    artist_found = limit_songs(artist_found)

    print("Extracting and cleaning lryics...")
    lyric_dict = get_lyrics(artist_found)

    print("Writing to pickle")
    write_pickle(artist, (lyric_dict, model_name) )

    return lyric_dict, model_name

In [18]:
ls_lyrics, model_name = get_and_save_artist_lyrics('Laura Stevenson')

Lyrics have already been created for this artist


In [21]:
from datasets import Dataset, DatasetDict
import numpy as np

def create_prelim_dataset(lyric_dict):
    lyrics = [v for k, v in lyric_dict.items()]

    my_dataset = Dataset.from_dict({'text' : lyrics})
    currLen = len(my_dataset)

    train_percentage = 0.85
    validation_percentage = 0.15
    test_percentage = 00

    train, valid , test = np.split(lyrics, [int(currLen*train_percentage), int(currLen*(train_percentage + validation_percentage))])

    datasets = DatasetDict(
                {
                    'train' : Dataset.from_dict({'text': train }),
                    'valid' : Dataset.from_dict({'text' : valid}),
                    'test' : Dataset.from_dict({'text' : test})
                })
    return datasets

ls_datasets = create_prelim_dataset(ls_lyrics)
# ls_datasets['train'][0]
# ls_datasets['train'][0]


In [30]:
def save_datasets(ls_datasets, model_name):
    folder_path = f'./models/{model_name}'
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    new_file = folder_path + '/' + 'datasets.p'

    with open(new_file, 'wb') as handle:
        pickle.dump(ls_datasets, handle)
    return

save_datasets(ls_datasets, model_name)

In [29]:
model_name

'laura-stevenson'

In [13]:
import pathlib
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir=pathlib.Path('cache').resolve())

: 

: 

In [9]:
import requests
requests.get('https://genius.com/Laura-stevenson-living-room-ny-lyrics').text



'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Genius</title>\n    <style>*{box-sizing:border-box}body{width:100%;margin:0;color:#fff;background:#000;font-family:Helvetica Neue,Arial,Helvetica,Geneva,sans-serif;font-size:20px;font-weight:700}a,a:active,a:focus{color:#3d85c6;text-decoration:none}a:hover{border-bottom:1px dotted}header{background-color:#ffff64;padding:10px;color:#000}header img{height:20px;display:block;margin:0 auto}main{max-width:860px;margin:0 auto}.text{text-align:center}.alert{margin:40px 20px}h1{font-size:7.5rem;text-transform:uppercase;font-weight:700;margin:0}@media screen and (max-width:420px){h1{font-size:3.5rem}}.distractions{margin:3rem auto 0;width:100%;max-width:700px}.iframe-container{width:100%;height:0;padding-top:56.25%;position:relative}.iframe-container iframe{max-width:100%;max-he

In [449]:
import itertools

def regroup_text(examples, block_size):
    combined = {k : list(itertools.chain.from_iterable(examples[k])) for k in examples.keys()}
    combined['input_ids']

    combined_size = len(combined['input_ids']) // block_size * block_size
    test_arr = [i for i in range(combined_size)]
    combined_size // block_size

    new_dict = {}
    for k, v in combined.items():
        val = []
        for i in range(combined_size // block_size):
            val.append(v[i * block_size : i * block_size + block_size])
        new_dict[k] = val

    new_dict['labels'] = new_dict['input_ids'].copy()

    return new_dict

In [450]:
from functools import partial

def tokenize_function(examples):
    return tokenizer(examples['text'])


def create_lm_dataset(prelim_dataset):
    int_datasets = prelim_dataset.map(tokenize_function, batched = True, remove_columns = ['text'])
    # [tokenizer.decode(i) for i in test['train'][0]['input_ids']]

    block_size = int(tokenizer.model_max_length / 4)

    regroup_texts_fn = partial(regroup_text, block_size = block_size)

    lm_datasets = int_datasets.map(
                    regroup_texts_fn,
                    batched = True,
                    batch_size = 100,
                    num_proc = 1
    )
    return lm_datasets

lm_datasets = create_lm_dataset(ls_datasets)

100%|██████████| 1/1 [00:00<00:00, 293.53ba/s]


In [451]:
import random
num_train_epochs = 30

trainer_state_path = f'{model_name}/trainer_state.json'
if os.path.isfile(trainer_state_path):
    f = open (trainer_state_path, "r")
    trainer_state = json.loads(f.read()) 
    f.close()
    epoch = trainer_state['epoch']
    num_train_epochs += epoch

seed_data = random.randint(0,2**32-1)

training_args = TrainingArguments(
    f"output/{model_name}",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=1.372e-4,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    save_total_limit=10,
    save_strategy='epoch',
    save_steps=1,
    report_to=None,
    seed=seed_data,
    logging_steps=5,
    do_eval=True,
    eval_steps=1,
    load_best_model_at_end=True
    # disable_tqdm=True
    # load_best_model_at_end=True
)

In [313]:
trainer = Trainer(
    model=model,
    # tokenizer=tokenizer,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)

['I',
 ' want',
 ' to',
 ' feel',
 ' you',
 ' restless',
 '\n',
 'I',
 ' want',
 ' to',
 ' wake',
 ' up',
 ' from',
 ' it',
 '\n',
 'I',
 ' want',
 ' to',
 ' see',
 ' you',
 ' stare',
 ' at',
 ' ceilings',
 ' until',
 ' you',
 ' fall',
 ' back',
 ' to',
 ' sleep',
 '\n',
 'I',
 ' want',
 ' to',
 ' feel',
 ' you',
 ' restless',
 '\n',
 'I',
 ' want',
 ' to',
 ' wake',
 ' up',
 ' from',
 ' it',
 '\n',
 'I',
 ' want',
 ' to',
 ' see',
 ' you',
 ' stare',
 ' at',
 ' ceilings',
 ' until',
 ' you',
 ' fall',
 ' back',
 ' to',
 ' sleep',
 '\n',
 'In',
 ' the',
 ' waiting',
 ' room',
 '\n',
 'In',
 ' the',
 ' waiting',
 ' room',
 '\n',
 'In',
 ' the',
 ' waiting',
 ' room',
 '\n',
 'It',
 '�',
 '�',
 's',
 ' a',
 ' little',
 ' stuff',
 'y',
 ' in',
 ' here',
 '\n',
 'I',
 ' want',
 ' to',
 ' wr',
 'ing',
 ' my',
 ' hands',
 ' in',
 ' open',
 ' elev',
 'ators',
 '\n',
 'A',
 ' shaky',
 ' finger',
 ',',
 ' I',
 ' am',
 ' pushing',
 ' the',
 ' door',
 ' open',
 ' button',
 '\n',
 'I',
 ' want',
 

In [193]:
ls_datasets['train'][0]

{'text': 'I want to feel you restless\nI want to wake up from it\nI want to see you stare at ceilings until you fall back to sleep\nI want to feel you restless\nI want to wake up from it\nI want to see you stare at ceilings until you fall back to sleep\nIn the waiting room\nIn the waiting room\nIn the waiting room\nIt’s a little stuffy in here\nI want to wring my hands in open elevators\nA shaky finger, I am pushing the door open button\nI want to feel the urge to press against my neighbors while I keep you in my mind\nI keep you in my mind all of the time here\nIn the waiting room\nIn the waiting room\nIn the waiting room\nOne is moving, one is standing\nIn the waiting room\nIn the waiting room\nIn the waiting room\nWonder where we’ll land\nWill we fall straight into routine?\nOr we’ll be strangers for a week, stir-crazy, lazy until I leave again\nThen I jump out of my skin until you tell me where I am\nLiving Room, New York\nLiving Room, New York\nI’ll give an arm just to hear you in

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])


In [10]:
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name) # np.log(1000)
# model.resize_token_embeddings(len(tokenizer))

Downloading: 100%|██████████| 665/665 [00:00<00:00, 193kB/s]
Downloading: 100%|██████████| 548M/548M [00:41<00:00, 13.2MB/s] 


: 

: 

In [111]:
genius.lyrics(swift2[0]['url'])

HTTPError: [Errno 403] 403 Client Error: Forbidden for url: https://api.genius.com/songs/https://genius.com/Taylor-swift-all-too-well-10-minute-version-taylors-version-from-the-vault-lyrics?text_format=plain
Action forbidden for current scope

In [102]:
import re


all too well


In [66]:

# genius.lyrics(song_url=url)

919
578


In [75]:
least_popular_song = genius.search_song(swiftsongs[0]['title'], 'Taylor Swift')
least_popular_song

Searching for "All Too Well (10 Minute Version) (Taylor’s Version) [From the Vault]" by Taylor Swift...
Done.


Song(id, artist, ...)

In [83]:
songs = genius.search_songs('Begin Again Andy Shauf')
songs['hits'][0]

{'highlights': [],
 'index': 'song',
 'type': 'song',
 'result': {'annotation_count': 10,
  'api_path': '/songs/2885745',
  'artist_names': 'Andy Shauf',
  'full_title': 'Begin Again by\xa0Andy\xa0Shauf',
  'header_image_thumbnail_url': 'https://images.genius.com/0d031f033e938b07532758a7e2d6e29f.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/0d031f033e938b07532758a7e2d6e29f.1000x1000x1.jpg',
  'id': 2885745,
  'language': 'en',
  'lyrics_owner_id': 1684296,
  'lyrics_state': 'complete',
  'path': '/Andy-shauf-begin-again-lyrics',
  'pyongs_count': None,
  'relationships_index_url': 'https://genius.com/Andy-shauf-begin-again-sample',
  'release_date_components': {'year': 2016, 'month': 5, 'day': 20},
  'release_date_for_display': 'May 20, 2016',
  'song_art_image_thumbnail_url': 'https://images.genius.com/0d031f033e938b07532758a7e2d6e29f.300x300x1.jpg',
  'song_art_image_url': 'https://images.genius.com/0d031f033e938b07532758a7e2d6e29f.1000x1000x1.jpg',
  'stats': {'un

In [70]:
swift2[0]

{'annotation_count': 38,
 'api_path': '/songs/7076626',
 'artist_names': 'Taylor Swift',
 'full_title': "All Too Well (10 Minute Version) (Taylor's Version) [From the Vault] by\xa0Taylor\xa0Swift",
 'header_image_thumbnail_url': 'https://images.genius.com/aac8ea3f13ae887a7f1fd9cdd451374e.300x300x1.png',
 'header_image_url': 'https://images.genius.com/aac8ea3f13ae887a7f1fd9cdd451374e.1000x1000x1.png',
 'id': 7076626,
 'language': 'en',
 'lyrics_owner_id': 12603744,
 'lyrics_state': 'complete',
 'path': '/Taylor-swift-all-too-well-10-minute-version-taylors-version-from-the-vault-lyrics',
 'pyongs_count': 211,
 'relationships_index_url': 'https://genius.com/Taylor-swift-all-too-well-10-minute-version-taylors-version-from-the-vault-sample',
 'release_date_components': {'year': 2021, 'month': 11, 'day': 12},
 'release_date_for_display': 'November 12, 2021',
 'song_art_image_thumbnail_url': 'https://images.genius.com/9dd4ba749dd51d39d7b56b67b9cc3777.300x300x1.jpg',
 'song_art_image_url': 'ht